# prédiction de type en fct date et lieu 

Après ces essais infructueux nous avons tenté de mettre en place un modèle de prédiction de type de crime en fonction de la date et de la position. Nous avons donc utilisé le modèle KNN classifier.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd 
import warnings 
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import random 
import folium
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
random.seed(42)
import os

In [3]:
df = pd.read_csv('../data/crime_in_la.csv')

In [4]:
df = df[df['Premis Desc'].notna()]
df = df[(df['LAT']!=0) & (df['LON']!=0)]
df['date'] = pd.to_datetime(df['DATE OCC'], format='%d/%m/%Y')
df.set_index('date', inplace=True)
df_f = df[["LAT","LON","Crm Cd"]]
df_f["Crm Cd"] = df_f["Crm Cd"].astype(str)

df_f = df_f.head(10000)

# Train begin on the 01/01/2022
date = '01-01-2022'

# train = df_f[df_f.index < date].copy()
# test = df_f[df_f.index >= date].copy()
train = df_f.head(8000).copy()
test = df_f.tail(2000).copy()

def get_data(df, labels= None):
    """
    Fonction permettant de créer les features à partir de la date

    Parameters
    ----------
    df : DataFrame
        DataFrame contenant les données
    labels : DataFrame, optional
        Noms des labels dans le dataframe
    
    Returns
    -------
    X : DataFrame
        DataFrame contenant les features
    y : DataFrame, optional
        DataFrame contenant les labels
    """
    df['date'] = df.index
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.weekofyear
    

    X = df[['dayofweek','quarter','month','year', 'dayofyear','dayofmonth','weekofyear',"LAT","LON"]]
    if labels:
        y = df[labels]
        return X, y
    return X

X_train, y_train = get_data(train, labels=["Crm Cd"])
X_test, y_test = get_data(test, labels=["Crm Cd"])

In [5]:
from geopy.distance import geodesic as GD
def from_geo_to_m(lon1,lon2,lat1,lat2) :
    """
    Conversion de coordonnées géographiques en mètres
    
    Parameters
    ----------
    lon1 : float
        Longitude du point 1
    lon2 : float
        Longitude du point 2
    lat1 : float
        Latitude du point 1
    lat2 : float
        Latitude du point 2
    
    Returns
    -------
    float
        Distance en mètres entre les deux points
    """
    return GD((lat1,lon1),(lat2,lon2)).m

In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

model = KNeighborsClassifier(
    n_neighbors=100, 
    weights='distance', 
    algorithm='ball_tree', 
    leaf_size=30, 
    p=2, 
    metric='minkowski', 
    metric_params=None, 
    n_jobs=None)

model.fit(X_train, y_train)

y_pred_train = model.predict(X_train)
print(f"Accuracy : {accuracy_score(y_train, y_pred_train)}")

y_pred = model.predict(X_test)
print(f"Accuracy : {accuracy_score(y_test, y_pred)}")

Accuracy : 0.959
Accuracy : 0.1075


L'accuracy sur le train est très élevée (0.95) mais sur le test elle est très faible (0.10). On est donc surement face à un problème de sur-apprentissage.